# TV

In [16]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from datetime import datetime,timedelta
import plotly.express as px
from numba import njit

In [17]:
import vectorbtpro as vbt
vbt.settings.set_theme("dark")

In [18]:
data = vbt.TVData.fetch(
    'NASDAQ:GOOG',
    timeframe = '1 day',
)
print(type(data.get()))

data.run("rsi",14).rsi
# vbt.EMA.run(close = data['Close'],length=14)
# data
# data.index = pd.to_datetime(data.index.date)
# start_date = '2017-09-27' 
# end_date = '2023-03-31'
# mask  = (data.index > start_date) & (data.index <= end_date)
# # data = data.loc[mask]
# data = data.round(2)
# data
len(data.get().loc[:,"Close"])
len(data.get()["Close"])

<class 'pandas.core.frame.DataFrame'>


2276

In [19]:
df = pd.DataFrame(data.get())
# fast_ema = data.run("ema", 10, hide_params=True)
ema = vbt.IF.from_pandas_ta('EMA')
# ema1 = EMA.run(df['Close'], length = 10).ema
# fast_ema.real_crossed_above(20)
a1 = ema.run(df['Close'], length = 10).ema
a2 = ema.run(df['Close'], length = 20).ema
a2.vbt.crossed_above(a1)
# df['Entry'] = df[df['EMA35'] > df['EMA50']]
# ema2 = vbt.IF.from_pandas_ta('EMA')
# ema.run(df['Close'], length = 20).ema
# fast_ema, slow_ema  = vbt.ema.run(close=df['Close'], length = 35).ema, vbt.ema.run(close=df['Close'], length = 50).ema
# entries = fast_ema.vbt.crossed_above(slow_ema)


datetime
2014-03-27 13:30:00+00:00    False
2014-03-28 13:30:00+00:00    False
2014-03-31 13:30:00+00:00    False
2014-04-01 13:30:00+00:00    False
2014-04-02 13:30:00+00:00    False
                             ...  
2023-04-04 13:30:00+00:00    False
2023-04-05 13:30:00+00:00    False
2023-04-06 13:30:00+00:00    False
2023-04-10 13:30:00+00:00    False
2023-04-11 13:30:00+00:00    False
Length: 2276, dtype: bool

In [20]:
# pf = vbt.PF.from_random_signals(
#     data, 
#     n=1,
#     tp_stop=0.5, 
#     sl_stop=0.1
# )
# pf.trade_history

# Check Indicator

In [21]:
ind = pd.DataFrame()
print(ind.ta.indicators())
# vbt.IF.list_indicators("*ma")

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

# EMA Function


In [22]:
# data, , startdate, enddate, , , , , ,, , , , , , , , , , equal,length
def EMA_CrossOver(symbol = None, timeframe = '1 day', sl_stop = None, tp_stop = None, tsl_stop = None, size = 1, max_size = None, init_cash = 100,
                  fast_length = 15, slow_length = 50, start_date = None, end_date = None):
    ''' 
    symbol = '{Market}:{Stock}'
    start_date = '{Year}-{Month}-{Day}' 
    end_date = '{Year}-{Month}-{Day}'
    slow_length = slow_ema
    fast_length = fast_ema
    tp_stop, sl_stop, tsl_stop = percentage / 100
    '''
    data = vbt.TVData.fetch(symbol, timeframe = timeframe)    
    #initialize dataFrame
    df = pd.DataFrame(data.get())
    # df = df.round(2)
    df.index = pd.to_datetime(df.index.date)
   
    # EMA
    ema = vbt.IF.from_pandas_ta('EMA')
    fast_ema, slow_ema  = ema.run(close = df['Close'], length = fast_length).ema, ema.run(close = df['Close'], length = slow_length).ema
    entries = fast_ema.vbt.crossed_above(slow_ema)
    exits = fast_ema.vbt.crossed_below(slow_ema)

    # Datetime
    if(start_date is not None and end_date is not None):
        mask  = (df.index >= start_date) & (df.index <= end_date)
        df = df.loc[mask]

    pf = vbt.Portfolio.from_signals(open = df["Open"], close = df["Close"], high = df["High"], low = df["Low"], 
                                    entries = entries, exits = exits, sl_stop = sl_stop, tp_stop = tp_stop, tsl_stop = tsl_stop,
                                    size = size, min_size = 1, max_size = max_size, init_cash = init_cash, price = 'NextOpen', stop_entry_price = 'FillPrice')
    return pf

In [23]:
# EMA_CrossOver(symbol = 'NASDAQ:GOOG', init_cash = 1000, start_date = '2017-09-27' ,end_date = '2023-02-25', tp_stop = 0.1).stats(settings=dict(freq='d'))
pf = EMA_CrossOver(symbol = 'NASDAQ:GOOG', init_cash = 1000, slow_length = 200, start_date = '2018-03-21', fast_length = 50)
# pf.orders.records_readable
# pf[ df.index == '2017-09-29']
# pf.orders.records_readable
#pf = pd.DataFrame(pf.orders.records_readable)
# 
pf.stats()
# help(EMA_CrossOver)
# pf.plot().show()
# pf.trade_history

/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/base/wrapping.py:961: UserWarning: Couldn't parse the frequency of index. Pass it as `freq` or de

Start                         2014-03-27 00:00:00
End                           2023-04-11 00:00:00
Period                                       2276
Start Value                                1000.0
Min Value                                996.3425
Max Value                               1098.4755
End Value                               1064.8815
Total Return [%]                          6.48815
Benchmark Return [%]                   281.071354
Total Time Exposure [%]                 69.815466
Max Gross Exposure [%]                  13.719833
Max Drawdown [%]                         3.248912
Max Drawdown Duration                       364.0
Total Orders                                    8
Total Fees Paid                               0.0
Total Trades                                    4
Win Rate [%]                                 50.0
Best Trade [%]                          69.347784
Worst Trade [%]                         -2.993051
Avg Winning Trade [%]                   64.800874


In [24]:
# EMA_CrossOver(symbol = 'NASDAQ:GOOG', init_cash = 1000, start_date = '2017-09-27' ,end_date = '2023-02-25', tp_stop = 0.1).stats(settings=dict(freq='d'))
pf = EMA_CrossOver(symbol = 'NASDAQ:GOOG', init_cash = 1000, start_date = '2017-09-27' ,end_date = '2023-02-25', tp_stop = 0.1, slow_length = 200, fast_length = 50)
# pf.orders.records_readable
# pf[ df.index == '2017-09-29']
# pf.orders.records_readable
#pf = pd.DataFrame(pf.orders.records_readable)

pf.stats()
# help(EMA_CrossOver)
# pf.plot().show()
# pf.trade_history

/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/base/wrapping.py:961: UserWarning: Couldn't parse the frequency of index. Pass it as `freq` or de

Start                         2014-03-27 00:00:00
End                           2023-04-11 00:00:00
Period                                       2276
Start Value                                1000.0
Min Value                                999.7665
Max Value                               1018.3706
End Value                               1018.3706
Total Return [%]                          1.83706
Benchmark Return [%]                    89.202638
Total Time Exposure [%]                  3.514938
Max Gross Exposure [%]                   7.354254
Max Drawdown [%]                         0.521617
Max Drawdown Duration                        18.0
Total Orders                                    6
Total Fees Paid                               0.0
Total Trades                                    3
Win Rate [%]                                100.0
Best Trade [%]                               10.0
Worst Trade [%]                              10.0
Avg Winning Trade [%]                        10.0


# Stochastic RSI

## stochRSI = (RSI – Lowest Low RSI) / (Highest High RSI – Lowest Low RSI)
1. overbought, oversold
- stochRSI > 0.8 = overbought
- stochRSI < 0.2 = oversold
2. trend
- stochRSI > 0.5 = uptrend
- stochRSI < 0.5 = downtrend

In [25]:
def StochRSI(symbol = None, timeframe = '1 day', sl_stop = None, tp_stop = None, tsl_stop = None, size = 1, max_size = None, init_cash = 100,
                  fast_length = 15, slow_length = 50, start_date = None, end_date = None):
    ''' 
    symbol = '{Market}:{Stock}'
    start_date = '{Year}-{Month}-{Day}' 
    end_date = '{Year}-{Month}-{Day}'
    slow_length = slow_ema
    fast_length = fast_ema
    tp_stop, sl_stop, tsl_stop = percentage / 100
    '''
    data = vbt.TVData.fetch(symbol, timeframe = timeframe)    
    #initialize dataFrame
    df = pd.DataFrame(data.get())
    # df = df.round(2)
    df.index = pd.to_datetime(df.index.date)
   
    # StochRSI
    stochrsi = vbt.IF.from_pandas_ta('STOCHRSI')
    fast_ema, slow_ema  = stochrsi.run(close = df['Close'], length = fast_length).stochrsi, stochrsi.run(close = df['Close'], length = slow_length).stochrsi
    entries = fast_ema.vbt.crossed_above(slow_ema)
    exits = fast_ema.vbt.crossed_below(slow_ema)

    # Datetime
    if(start_date is not None and end_date is not None):
        mask  = (df.index >= start_date) & (df.index <= end_date)
        df = df.loc[mask]

    pf = vbt.Portfolio.from_signals(open = df["Open"], close = df["Close"], high = df["High"], low = df["Low"], 
                                    entries = entries, exits = exits, sl_stop = sl_stop, tp_stop = tp_stop, tsl_stop = tsl_stop,
                                    size = size, min_size = 1, max_size = max_size, init_cash = init_cash, price = 'NextOpen', stop_entry_price = 'FillPrice')
    return pf

In [26]:
data = vbt.TVData.fetch('NASDAQ:GOOG', timeframe = '1 day')    
#initialize dataFrame
df = pd.DataFrame(data.get())
# df = df.round(2)
df.index = pd.to_datetime(df.index.date)
ema = vbt.IF.from_pandas_ta('EMA')

# StochRSI
stochrsi = vbt.IF.from_pandas_ta('stochrsi')
# fast_ema, slow_ema  = stochrsi.run(close = df['Close'], length = fast_length).stochrsi, stochrsi.run(close = df['Close'], length = slow_length).stochrsi
vbt.phelp(stochrsi.run)

STOCHRSI.run(
    close,
    length=Default(value=None),
    rsi_length=Default(value=None),
    k=Default(value=None),
    d=Default(value=None),
    mamode=Default(value=None),
    offset=Default(value=None),
    short_name='stochrsi',
    hide_params=None,
    hide_default=True,
    **kwargs
):
    Run `STOCHRSI` indicator.
    
    * Inputs: `close`
    * Parameters: `length`, `rsi_length`, `k`, `d`, `mamode`, `offset`
    * Outputs: `stochrsik`, `stochrsid`
    
    Pass a list of parameter names as `hide_params` to hide their column levels, or True to hide all.
    Set `hide_default` to False to show the column levels of the parameters with a default value.
    
    Other keyword arguments are passed to `STOCHRSI.run_pipeline`.
